# <center>Aprendizaje Automático Avanzado: Ejercicio final</center>
### <center>Autor: Julen Ercibengoa</center>

## 1. Contexto

Actualmente estoy trabajando en la empresa **Tekniker** la cual está involucrada en el proyecto GAIA. Este es un proyecto que pretende dar apoyo a los servicios de extinción de incendios forestales mediante herramientas de prevención y detección temprana utilizando la Inteligencia Artificial.

Mi trabajo consiste en crear un modelo de Aprendizaje Automático que prediga el riesgo de incendio forestal a nivel nacional diariamente. Es decir, el objetivo final del trabajo (el cual es mi Trabajo de Fin de Máster), es generar un modelo que con los datos disponibles hasta hoy, genere un mapa de mañana de riesgo de incendio forestal en toda España. 

Para conseguir ese objetivo hacen falta varios pasos, y este trabajo será uno de ellos. En este trabajo haremos una primera prueba y generaremos un predictor de riesgo de incendio forestal que prediga el riesgo de incendio forestal mensual. Además, haremos varios análisis para decidir qué variable son relevantes para el modelo.

## 2. Base de Datos

Para predecir el riesgo de incendio forestal hay que generar una base de datos con la cual entrenar un modelo de Inteligencia Artificial. En el estado del arte [(Ver el análisis del estado del arte)](SOTA.pdf) se sigue una metodología común que vamos a utilizar: se divide la región a analizar en celdas espacio-temporales y cada celda tiene asignadas variables tabulares. Para generar esa base de datos, vamos a utilizar las siguientes fuentes de datos:
- <u>Datos históricos de fuegos</u>: base de datos de [EFFIS](https://forest-fire.emergency.copernicus.eu/)
- <u>Datos del tipo de terreno</u>: base de datos [Corine Land Cover](https://land.copernicus.eu/en/products/corine-land-cover)
- <u>Datos DEM (Digital Elevation Model)</u>: base de datos de [EU DEM](https://portal.opentopography.org/raster?opentopoID=OTSDEM.032021.4326.3&minX=-17.18261718749999&minY=31.097629956393973&maxX=15.952148437499998&maxY=45.43508099838451)
- <u>Distancia a carreteras</u>, <u>distancia a rios</u> y <u>densidad poblacional</u>: [WorldPop](https://www.worldpop.org/)
- <u>Distancia a vías de tren</u>: [OpenStreetMap](https://www.openstreetmap.org/#map=6/40.01/-2.49)
- <u>Proporción de natura2000</u>: [Gobierno de España](https://www.miteco.gob.es/es/biodiversidad/servicios/banco-datos-naturaleza/informacion-disponible/rednatura_2000_desc.html)

Aunque ahora vayamos a visualizar estas fuentes de datos, para construir el "datacube" (la base de datos que tiene toda la información homogeneizada en el tiempo y en el espacio) se ha utilizado el software QGIS, que sirve para hacer análisis de datos espaciales.

### 2.1 Datos históricos de fuegos: EFFIS

In [6]:
import geopandas as gpd
gdf_fires = gpd.read_file("../data/Variables/EFFIS_historical_postprocessed.json")

In [13]:
import contextily as ctx
import matplotlib.pyplot as plt
gdf_fires = gdf_fires.to_crs(epsg=3857)
minx, miny, maxx, maxy = gdf_fires.total_bounds
ax = gdf_fires.plot(column="area_ha", legend=True, figsize=(12,12), cmap="plasma", legend_kwds={'label': "Área quemada (ha)", 'orientation': "horizontal", 'shrink': 0.5, 'aspect': 50, 'pad': 0.01})
ax.set_xlim([-1.25*10**6, maxx])
ax.set_ylim([4.25*10**6, maxy])
ctx.add_basemap(ax, crs=gdf_fires.crs.to_string())
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Área quemada en España (2008-2024)", fontsize=20, pad=20)

plt.savefig("../data/figures/fires_plot.png", bbox_inches="tight", dpi=300)
plt.clf()


<Figure size 1200x1200 with 0 Axes>

<div style="text-align: center;">
    <img src="../data/figures/fires_plot.png" alt="Área quemada en España (2008-2024)" width="800">
</div>

### 2.2 Datos del tipo de terreno: Corine Land Cover